In [3]:
import re
import sys
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
%matplotlib inline

In [4]:
repos = pd.read_csv("repos5197.csv", encoding="utf-8")

In [5]:
awe_repos = pd.read_csv("awe_repos_100_in_cats.csv", encoding="utf-8")

In [6]:
awe_repos["repo_name"] = awe_repos["repo"].str.split("/").str.get(1)
awe_repos["source_name"] = awe_repos["source_repo"].str.split("/").str.get(1)
awe_repos["categories"] = awe_repos["categories"].str.replace("^\[", "").str.replace("\]$","")
def remove_name(x):
    cats = x["categories"].split(", ")
    #if cats[-1] != x["repo_name"]:
        #print(cats[-1], x["repo_name"])
    cats.pop()
    return ", ".join(cats)

awe_repos["categories2"] = awe_repos.apply(remove_name, axis=1)
awe_repos["categories3"] = awe_repos["source_name"] + ", " + awe_repos["categories2"]

In [7]:
COLS =  ['description',
'full_name',
'language',
'homepage',
'fork']

repos2 = repos[COLS]

In [8]:
#pd.merge(repos2, awe_repos, left_on=["full_name"], right_on=["repo"])
pd.merge(repos2, awe_repos, left_on=["full_name"], right_on=["repo"]).to_csv("repos_cats.csv", encoding="utf-8", index=False)

In [13]:
#repos2 = pd.merge(repos2, awe_repos, left_on=["full_name"], right_on=["repo"])

desc_text = pd.merge(repos2, awe_repos, left_on=["full_name"], right_on=["repo"]).drop_duplicates(["description", "full_name"]) \
    .dropna(subset = ['description'])
    #.dropna(subset = ['categories2'])

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)    
    
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(desc_text["description"])
#dtm_tfidf = tfidf_vectorizer.fit_transform(desc_text["categories2"])

print(dtm_tfidf.shape)    

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_topics=100, random_state=42)
lda_tfidf.fit(dtm_tfidf)

(4972, 575)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=100, perp_tol=0.1, random_state=42,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [14]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

print("\nTopics in LDA model:")
tf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lda_tfidf, tf_feature_names, 20)


Topics in LDA model:
Topic #0:
package code files file generate management better strings small pattern visual compile ide swift plug simple using time convert xcode
Topic #1:
beautiful notification alert ios view messages alternative standard information create makes easy written replacement just pattern list class plugin element
Topic #2:
free boilerplate functional scala support customizable fast storage layer maps local blocking integrate cross code mac iphone debugging pretty powered
Topic #3:
plugin jquery slider values different range types simple image visualization store web form information windows create notification alternative powerful standard
Topic #4:
transitions script protocol uses toolkit packages written url components using monitor unofficial elasticsearch docker color cloud getting little blocking let
Topic #5:
time audio dependency real stack visualization injection touch multi library framework swift lightweight built simple javascript android engine objective 

In [15]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
73     4.904653        1       1 -0.187981 -0.089603
39     3.460755        1       2 -0.225560 -0.200688
91     2.895139        1       3 -0.190205 -0.005286
59     2.636647        1       4 -0.180165 -0.128206
78     2.369380        1       5 -0.164975 -0.042886
81     2.365516        1       6 -0.156813 -0.058768
75     2.303308        1       7 -0.174149 -0.004312
7      2.279669        1       8 -0.148515  0.140419
0      2.210035        1       9 -0.161473  0.027706
28     2.091613        1      10 -0.182725 -0.029999
99     2.063691        1      11 -0.166986 -0.141330
47     1.961588        1      12 -0.164319 -0.086398
43     1.934264        1      13 -0.205685 -0.073115
86     1.901633        1      14 -0.177810  0.128408
97     1.891797        1      15 -0.158860 -0.064079
87     1.821861        1      16 -0.149828 -0.070249
37     1.790121        1      17 -0.168812 -0.150624
64     1.725705        1      18 -0.149516  0.014969
19     1.721551        1      19 -0.180057 -0.153555
44     1.715874        1      20 -0.162127  0.011474
20     1.663158        1      21 -0.136532  0.118097
85     1.661429        1      22 -0.129240  0.025855
52     1.653004        1      23 -0.180813  0.047127
31     1.573806        1      24 -0.142498  0.125215
6      1.501741        1      25 -0.132175 -0.071814
14     1.299487        1      26 -0.118893  0.094319
66     1.283151        1      27 -0.127703 -0.002865
46     1.256034        1      28 -0.112713  0.165224
49     1.250245        1      29 -0.161194 -0.084029
42     1.237253        1      30 -0.123006  0.050094
...         ...      ...     ...       ...       ...
29     0.376706        1      71  0.183168 -0.013152
16     0.334670        1      72  0.190736 -0.017347
12     0.334670        1      73  0.190736 -0.017347
21     0.334670        1      74  0.190736 -0.017347
4      0.334670        1      75  0.190736 -0.017347
83     0.334670        1      76  0.190736 -0.017347
89     0.334670        1      77  0.190736 -0.017347
80     0.334670        1      78  0.190736 -0.017348
70     0.334670        1      79  0.190736 -0.017347
22     0.334670        1      80  0.190735 -0.017349
98     0.334670        1      81  0.190736 -0.017347
68     0.334670        1      82  0.190736 -0.017347
71     0.334670        1      83  0.190736 -0.017347
65     0.334670        1      84  0.190736 -0.017347
62     0.334670        1      85  0.190736 -0.017347
60     0.334670        1      86  0.190736 -0.017347
55     0.334670        1      87  0.190736 -0.017347
54     0.334670        1      88  0.190736 -0.017347
51     0.334670        1      89  0.190736 -0.017348
48     0.334670        1      90  0.190736 -0.017347
24     0.334670        1      91  0.190736 -0.017347
45     0.334670        1      92  0.190736 -0.017347
41     0.334670        1      93  0.190736 -0.017348
95     0.334670        1      94  0.190736 -0.017347
32     0.334670        1      95  0.190736 -0.017347
79     0.334670        1      96  0.190736 -0.017347
30     0.334670        1      97  0.190736 -0.017347
27     0.334670        1      98  0.190736 -0.017348
26     0.334670        1      99  0.190736 -0.017348
88     0.334670        1     100  0.190736 -0.017347

[100 rows x 5 columns], topic_info=      Category        Freq            Term       Total  loglift  logprob
term                                                                    
253    Default  144.000000             ios  144.000000  30.0000  30.0000
332    Default  111.000000          native  111.000000  29.0000  29.0000
20     Default   73.000000             app   73.000000  28.0000  28.0000
413    Default  106.000000           react  106.000000  27.0000  27.0000
442    Default   85.000000            rust   85.000000  26.0000  26.0000
256    Default   68.000000      javascript   68.000000  25.0000  25.0000
467    Def

In [68]:
awe_repos["repo_name"] = awe_repos["repo"].str.split("/").str.get(1)
awe_repos["source_name"] = awe_repos["source_repo"].str.split("/").str.get(1)
awe_repos["categories"] = awe_repos["categories"].str.replace("^\[", "").str.replace("\]$","")
def remove_name(x):
    cats = x["categories"].split(", ")
    #if cats[-1] != x["repo_name"]:
        #print(cats[-1], x["repo_name"])
    cats.pop()
    return ", ".join(cats)

awe_repos["categories2"] = awe_repos.apply(remove_name, axis=1)
awe_repos["categories3"] = awe_repos["source_name"] + ", " + awe_repos["categories2"]

In [69]:
awe_repos

,categories,repo_url,source_repo,repo,items_count,repo_name,categories2,source_name
0,awesome-npm,https://github.com/sindresorhus/awesome-npm,sindresorhus/awesome-nodejs,sindresorhus/awesome-npm,438,awesome-npm,,awesome-nodejs
1,"Mad science, webtorrent",https://github.com/feross/webtorrent,sindresorhus/awesome-nodejs,feross/webtorrent,438,webtorrent,Mad science,awesome-nodejs
2,"Mad science, GitTorrent",https://github.com/cjb/GitTorrent,sindresorhus/awesome-nodejs,cjb/GitTorrent,438,GitTorrent,Mad science,awesome-nodejs
3,"Mad science, peerflix",https://github.com/mafintosh/peerflix,sindresorhus/awesome-nodejs,mafintosh/peerflix,438,peerflix,Mad science,awesome-nodejs
4,"Mad science, ipfs",https://github.com/ipfs/js-ipfs,sindresorhus/awesome-nodejs,ipfs/js-ipfs,438,js-ipfs,Mad science,awesome-nodejs
5,"Mad science, peerwiki",https://github.com/mafintosh/peerwiki,sindresorhus/awesome-nodejs,mafintosh/peerwiki,438,peerwiki,Mad science,awesome-nodejs
6,"Mad science, peercast",https://github.com/mafintosh/peercast,sindresorhus/awesome-nodejs,mafintosh/peercast,438,peercast,Mad science,awesome-nodejs
7,"Mad science, turf",https://github.com/Turfjs/turf,sindresorhus/awesome-nodejs,Turfjs/turf,438,turf,Mad science,awesome-nodejs
8,"Mad science, webcat",https://github.com/mafintosh/webcat,sindresorhus/awesome-nodejs,mafintosh/webcat,438,webcat,Mad science,awesome-nodejs
9,"Mad science, js-git",https://github.com/creationix/js-git,sindresorhus/awesome-nodejs,creationix/js-git,438,js-git,Mad science,awesome-nodejs


In [70]:
def remove_name(x):
    cats = x["categories"].split(", ")
    #if cats[-1] != x["repo_name"]:
        #print(cats[-1], x["repo_name"])
    cats.pop()
    return ", ".join(cats)

awe_repos["categories2"] = awe_repos.apply(remove_name, axis=1)
awe_repos["categories3"] = awe_repos["source_name"] + ", " + awe_repos["categories2"]

In [77]:
awe_repos

,categories,repo_url,source_repo,repo,items_count,repo_name,categories2,source_name,categories3
0,awesome-npm,https://github.com/sindresorhus/awesome-npm,sindresorhus/awesome-nodejs,sindresorhus/awesome-npm,438,awesome-npm,,awesome-nodejs,"awesome-nodejs,"
1,"Mad science, webtorrent",https://github.com/feross/webtorrent,sindresorhus/awesome-nodejs,feross/webtorrent,438,webtorrent,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
2,"Mad science, GitTorrent",https://github.com/cjb/GitTorrent,sindresorhus/awesome-nodejs,cjb/GitTorrent,438,GitTorrent,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
3,"Mad science, peerflix",https://github.com/mafintosh/peerflix,sindresorhus/awesome-nodejs,mafintosh/peerflix,438,peerflix,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
4,"Mad science, ipfs",https://github.com/ipfs/js-ipfs,sindresorhus/awesome-nodejs,ipfs/js-ipfs,438,js-ipfs,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
5,"Mad science, peerwiki",https://github.com/mafintosh/peerwiki,sindresorhus/awesome-nodejs,mafintosh/peerwiki,438,peerwiki,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
6,"Mad science, peercast",https://github.com/mafintosh/peercast,sindresorhus/awesome-nodejs,mafintosh/peercast,438,peercast,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
7,"Mad science, turf",https://github.com/Turfjs/turf,sindresorhus/awesome-nodejs,Turfjs/turf,438,turf,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
8,"Mad science, webcat",https://github.com/mafintosh/webcat,sindresorhus/awesome-nodejs,mafintosh/webcat,438,webcat,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"
9,"Mad science, js-git",https://github.com/creationix/js-git,sindresorhus/awesome-nodejs,creationix/js-git,438,js-git,Mad science,awesome-nodejs,"awesome-nodejs, Mad science"


In [80]:
#awe_repos.groupby(["categories3"]).agg("count").sort_values(["categories"], ascending=False)
pd.merge(repos2, awe_repos, left_on=["full_name"], right_on=["repo"]).to_csv("repos_cats.csv", encoding="utf-8", index=False)